# Data processing

The water monitoring project at the Montreux Jazz has been going on since 2016. The data has been collected and treated by a variety of people since then. 

__Objective:__ Standardize the nomenclature from the different sampling years. Provide a model for storing and collecting data in the future.

__Purpose:__ Define the probability that a survey will exceed a threshold value within the period of the year defined by the survey results.

## Definitions

* colony: a circular discoloration of the media within a defined size and color range
* colony-count: the number of discolorations of the same hue for a media type
* media/medium: the environment that the water samples are placed in
* color: the observed color of the colony
* label: the assumed category of the color:
  * Bioindicator
  * Coliform
  * Other
* coef: the volume of water used for the sample
 
The purpose of the sampling is to identify colonies that appear in the media and classify them as one of the possible labels. The label of interest is _Bioindicators_, this represents the bacteria that are issue from the organism of interest. The organism in this case is people, the _Bioindicator_ is issue from fecal contaminants.

## Methods

The process requires collaborating with the data-manager(s) from the different project years and ensuring that the data from each year can be combined and interpreted together. The data for this collaboration is sotred in the _componentdata_ folder.

The relationship of previous label <---> new label is stored in a dictionary for the different possibilities of medium, color, label and coefficient. The new labels are applied to a data-frame.

The finsihed data (the result of the collaboration) is stored in the _end_ folder

## Sample data

The sample data is an example of the desired output per year. This includes the following parameters:

1. colony-count
2. label
3. location
4. coeficient*count
5. week number
6. day of year
7. is-jazz: boolean
8. rain fall in millimeters

In [8]:
import pandas as pd
import datetime as dt
import numpy as np

project = "Hackuarium do it together water quality sampling"
site_markers = {"SVT":"o", "VNX":"D", "MRD":"X"}
species_colors = { "Bioindicator":"dodgerblue", "Coliform":"magenta"}
marker_colors = {"SVT":"black", "VNX":"green", "MRD":"goldenrod"}
sites = ["SVT", "VNX", "MRD"]

In [10]:
df = pd.read_csv("data/end/survey_data_2020_2023.csv")
df['date'] = pd.to_datetime(df["date"])
df['week'] = df["date"].dt.isocalendar().week
df.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24


In [11]:
# translate colors
def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met"]
coliforms = ["Pink", "pink"]
other = ["Turquoise", "Turquoise slow", "other"]

df["label"] = df.color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x

media_names = {
    "ECC-A Card":"ECC-A",
    "new ECCA":"ECC-A",
    "E-coli side": "E coli",
    "ECC-side":"ECC",
    "selective":"Levine",
    "media":"EasyGel",
    "plus uv":"EasyGelPlus",
    "UVplus":"EasyGelPlus",
    "non-restrictive":"LB"
}

df["medium"] = df.media.apply(lambda x: translate_media(x, media_names))

In [12]:
df["medium"].unique()

array(['ECC-A', 'Double side ECC', 'Double side E coli', 'ECC', 'LB',
       'levine'], dtype=object)

In [13]:
df["label"].unique()

array(['Bioindicator', 'Other', 'Coliform', 'purple', 'mauve'],
      dtype=object)

In [14]:
df.coef.unique()

array([100.,   1.])

In [15]:
df.year.unique()

array([2023, 2022, 2020])

In [16]:
df.week.unique()

<IntegerArray>
[24, 25, 26, 27, 28, 29, 30, 31]
Length: 8, dtype: UInt32